In [ ]:
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

from utils.loadData import loadDataWithNews, loadData
from lstm.preprocessing import dataToTimeSeriesMultiDimensional, getDatasetsMultiDimensional, normalizeAtOnce
from lstm.model import getModel, evaluateModelQuick, evaluateFinal
from lstm.visualization import visualize_loss, show_batch, show_data_simple, showRegressionExampleMultiDimensionalKBest, show_heatmap
from lstm.callbacks import es_callback, modelckpt_callback
from keras.utils.vis_utils import plot_model

In [ ]:
df = loadData()

In [ ]:
features = df.loc[:, df.columns != 'Date']
features = features.loc[:, features.columns != 'Unnamed: 0']
features = features.loc[:, features.columns != 'Adj Close']
features.head()

In [ ]:
normalized_data, scaler = normalizeAtOnce(features)
show_data_simple(normalized_data[:,3], df["Date"])

In [ ]:
# Data correlation
show_heatmap(features)

In [ ]:
# pearson's correlation feature selection for numeric input and numeric output
X = normalized_data[:-1]
y = normalized_data[1:, 3]
# define feature selection
fs = SelectKBest(score_func=f_regression, k=4)
# apply feature selection
X_selected = fs.fit_transform(X, y)
print("Open High Low Close Volume sents")
print(normalized_data[10])
print(X_selected[10])

# Feature ranking:
# Close, High, Low, Open, negative, volume, neutral, positive

In [ ]:
config = {
    "past": 10,
    "future": 10,
    "neurons": 50,
    "batch_size": 128,
    "epochs": 50,
    "features": X_selected.shape[1]
}

dataset_train, dataset_val, x_test, y_test = getDatasetsMultiDimensional(config, X_selected, 0)


for batch in dataset_val.take(1):
    inputs, targets = batch
    show_batch(inputs[0, :, 0], targets[:config["future"]], config["past"])

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
model = getModel(config)
model.summary()

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(
    dataset_train,
    epochs=config["epochs"],
    validation_data=dataset_val,
    callbacks=[es_callback, modelckpt_callback],
)
visualize_loss(history)

In [ ]:
x, y = dataToTimeSeriesMultiDimensional(x_test, y_test[:,0], config["past"], config["future"])

In [ ]:
y_test.shape

In [ ]:
showRegressionExampleMultiDimensionalKBest(x[-25], y[-25], model, scaler, dim=5)

In [ ]:
evaluateFinal(getModel, dataset_train, dataset_val, x, y, config, "regression_with_news_features_chosen_new_k4")